# 뉴스기사 요약해보기

In [1]:
import nltk
nltk.download('stopwords')

import numpy as np
import pandas as pd
import os
import re
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from bs4 import BeautifulSoup 
from tensorflow.keras.preprocessing.text import Tokenizer 
from tensorflow.keras.preprocessing.sequence import pad_sequences
import urllib.request
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module='bs4')

[nltk_data] Downloading package stopwords to /aiffel/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Step 1. 데이터 수집하기
- [news_summary_more.csv](https://github.com/sunnysai12345/News_Summary)  
- 추상적 요약을 하는 경우에는 text를 본문, headlines를 이미 요약된 데이터로 삼아서 모델 학습하기. 추출적 요약을 하는 경우에는 text열만을 사용하기.

In [2]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/sunnysai12345/News_Summary/master/news_summary_more.csv", filename="news_summary_more.csv")
data = pd.read_csv('news_summary_more.csv', encoding='iso-8859-1')

data.sample(10)

,headlines,text
20972,"I'm working mother, wife; hope women get inspi...",Kareena Kapoor Khan has said she's a working m...
95363,MDMK's Vaiko sent to 15-day judicial custody f...,"MDMK leader V Gopalsamy, also known as Vaiko, ..."
7636,I now see it as a blessing: Katrina Kaif on he...,"Actress Katrina Kaif, while opening up about h..."
49651,Apple to start iPhone 6s production in India: ...,Technology giant Apple's Taiwanese contract ma...
88494,Against discriminatory things: Jacqueline on f...,"Jacqueline Fernandez, while talking about endo..."
9142,Wiped-out US hedge fund manager confesses his ...,A US-based hedge fund manager has apologised f...
89676,Scientists spot rare 'Dragon-skin' ice in Anta...,Scientists on an Antarctic expedition aboard a...
13302,"ISIS takes 700 refugees hostage in Syria, says...",The Islamic State attacked a refugee camp in S...
3897,What has been AITA's vision for tennis in last...,Former Indian tennis player Somdev Devvarman q...
4412,Priests shut Puri Jagannath Temple after tussl...,Odisha's Jagannath Temple was closed on Friday...


## Step 2. 데이터 전처리하기 (추상적 요약)

## Step 3. 어텐션 메커니즘 사용하기 (추상적 요약)

## Step 4. 실제 결과와 요약문 비교하기 (추상적 요약)

## Step 5. Summa을 이용해서 추출적 요약해보기

------

## 회고

## Reference